In [3]:
!pip install datasketch

  Using cached datasketch-1.6.4-py3-none-any.whl.metadata (5.8 kB)
Using cached datasketch-1.6.4-py3-none-any.whl (88 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd

from style_generation import get_style_genre
from first_n_words import get_first_n_words
from llm import ask_LLM
from kg_content import extract_kg_content
from minhash_vector import create_minhash_vector
from reconstruction_content import extract_reconstruction_content
from evaluate import evaluate_peformance

In [12]:
# Load the dataset from Hugging Face
dataset = pd.read_csv("dataset/ML-Arxiv-Papers.csv")

# Extract the 'train' split
#train_dataset = dataset["train"]

# Create lists for titles and abstracts
# titles = [entry['title'] for entry in train_dataset]
# abstracts = [entry['abstract'] for entry in train_dataset]

# Create a list with concatenated title and abstract for each sample
concatenated_texts = dataset['abstract'] #[f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]

API_KEY = "2356018b771d503cd9202a0a35195998bc3fabda6b1e6b379ee6ecc479f67401"

stop_len = 5000

all_kg_results = []
all_reconstruction_results = []
input_string_so_far_list = []


KeyError: 'abstracts'